In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# Training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Validation without augmentation (only rescaling)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [6]:
train = train_datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val = val_datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(32, 32),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [8]:
CNN_model = Sequential()
CNN_model.add(Input(shape=(32, 32, 3)))

CNN_model.add(Conv2D(16, (3, 3), activation='relu'))
CNN_model.add(Conv2D(32, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(BatchNormalization())

CNN_model.add(Conv2D(32, (3, 3), activation='relu'))
CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(BatchNormalization())

CNN_model.add(Flatten())
CNN_model.add(Dense(256,activation='relu'))
CNN_model.add(Dense(5, activation='softmax'))

CNN_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
CNN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5, 5, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       409,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 444,357 (1.70 MB)

 Trainable params: 444,165 (1.69 MB)

 Non-trainable params: 192 (768.00 B)

In [10]:
CNN_model.fit(train, validation_data=val, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 55s 470ms/step - accuracy: 0.4213 - loss: 1.6391 - val_accuracy: 0.4000 - val_loss: 1.4683
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 26s 234ms/step - accuracy: 0.5342 - loss: 1.2989 - val_accuracy: 0.3581 - val_loss: 1.5123
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 230ms/step - accuracy: 0.5556 - loss: 1.1641 - val_accuracy: 0.4640 - val_loss: 1.3028
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 26s 240ms/step - accuracy: 0.6027 - loss: 1.0407 - val_accuracy: 0.5070 - val_loss: 1.1961
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 26s 235ms/step - accuracy: 0.6214 - loss: 0.9849 - val_accuracy: 0.5698 - val_loss: 1.2205
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 233ms/step - accuracy: 0.6397 - loss: 0.9418 - val_accuracy: 0.6000 - val_loss: 0.9972
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 234ms/step - accuracy: 0.6513 - loss: 0.9202 - val_accuracy: 0.5953 - val_loss: 1.1543
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 25s 232ms/step - accuracy: 0.6630 - loss: 0